In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from numba import jit 

In [2]:
@jit(nopython=True)
def legendre(n,x):
    leg = 1.
    if n == 0:
        leg = 1
    elif n==1:
        leg = x
    else:
        leg_down1 = x; leg_down2 = 1.
        for i in range(2,n+1):
            leg = (2*i-1)*x*leg_down1/i - (i-1)*leg_down2/i
            leg_down2 = leg_down1
            leg_down1 = leg

    return leg

@jit(nopython=True)
def dlegendre(n,x):
    dleg = 0.
    if n == 0:
        dleg = 0.
    elif n == 1:
        dleg = 1.
    else:
        leg_down1 = x; leg_down2 = 1.
        dleg_down1 = 1.; dleg_down2 = 0.
        for i in range(2,n+1):
            leg = (2*i-1)*x*leg_down1/i - (i-1)*leg_down2/i
            dleg = dleg_down2 + (2*i-1)*leg_down1
            leg_down2 = leg_down1
            leg_down1 = leg
            dleg_down2 = dleg_down1
            dleg_down1 = dleg

    return dleg

@jit(nopython=True)
def gauss_legendre_lobatto(length:int):
    #// define some constants
    tolerance = 4.0 * 1.0e-6
    nnewton_iter = 100
    pi = np.pi
    
    # allocate space
    n = length - 1; #// order of polynomial
    x = np.array([0.0 for i in range(length)])
    w = x * 1.
    if n == 1:
        x[0] = -1.; x[1] = 1.
        w[0] = 1.; w[1] = 1.
    else:
        leg = 0.; dleg = 0.; delta = 0.
        # set end points
        x[0]   = -1.0; x[n] = 1.
        w[0]   =  2./(n*(n+1.)); w[n] =  2./(n*(n+1.))

        for i in range(1,(n+1)//2):
            #// initial guess from an approximate form given by SV Parter (1999)
            x[i] = -np.cos( (i+0.25)*pi/n  - 3/(8*n*pi*(i+0.25)))

            #// newton iteration
            for j in range(nnewton_iter):
                leg = legendre(n+1,x[i]) - legendre(n-1,x[i])
                dleg = dlegendre(n+1,x[i]) - dlegendre(n-1,x[i])
                delta = -leg/dleg
                x[i] += delta
                if (np.abs(delta) <= tolerance * np.abs(x[i]) ): break
            
            x[n-i] = - x[i]
            leg = legendre(n, x[i])
            w[i] = 2./(n*(n+1.)*leg*leg)
            w[n-i] = w[i]

        if n %2 == 0 :
            x[n//2] = 0.
            leg = legendre(n, 0.0)
            w[n//2]  = 2./(n*(n+1.)*leg*leg)
    

    return x,w

@jit(nopython=True)
def lagrange_poly(xi,xctrl):
    nctrl = len(xctrl)
    hprime = np.array([0.0 for i in range(nctrl)])
    h = hprime * 1.0

    #! note: this routine is hit pretty hard by the mesher, optimizing the loops here will be beneficial
    for dgr in range(nctrl):
        prod1 = 1.; prod2 = 1.

        #// lagrangian interpolants
        x0 = xctrl[dgr]
        for i in range(nctrl):
            if i != dgr:
                x = xctrl[i]
                prod1 = prod1*(xi-x)
                prod2 = prod2*(x0-x)

        #//! takes inverse to avoid additional divisions
        #//! (multiplications are cheaper than divisions)
        prod2_inv = 1. / prod2
        h[dgr] = prod1 * prod2_inv

        #// first derivatives
        s = 0.0
        for i in range(nctrl):
            if i != dgr :
                prod3 = 1.0
                for j in range(nctrl):
                    if j != dgr and j != i:
                        prod3 = prod3*(xi-xctrl[j])
                s = s + prod3
        hprime[dgr] = s * prod2_inv
    

    return h,hprime


def init_gll_arrays(NGLLX):

    # gll points and weights
    xgll,wxgll = gauss_legendre_lobatto(NGLLX)

    #// derivative, hprimex[i][j] = l_j'(xi_i)
    hprimex = np.zeros((NGLLX,NGLLX))
    for i in range(NGLLX):
        _,hprimex[i,:] = lagrange_poly(xgll[i],xgll)

    #// hprimex_wgllx[i,j] = hprime_x[i,j] * wx[i] 
    hprimex_wgllx = np.zeros((NGLLX,NGLLX))
    for i in range(NGLLX):
        for j in range(NGLLX):
            hprimex_wgllx[i,j] = hprimex[i,j] * wxgll[i]


    return xgll,wxgll,hprimex,hprimex_wgllx



In [3]:
@jit(nopython=True)
def get_Q_sls_model(Q):

    y_sls_ref = np.array([1.93044501, 1.64217132, 1.73606189, 1.42826439, 1.66934129])
    w_sls_ref = np.array([4.71238898e-02, 6.63370885e-01, 9.42477796e+00, 1.14672436e+02,1.05597079e+03])
    dy = y_sls_ref * 0 
    y = y_sls_ref * 0
    NSLS = len(y)

    for i in range(NSLS):
        y[i] = y_sls_ref[i] / Q 
    dy[0] = 1. + 0.5 * y[0]

    for i in range(1,NSLS):
        dy[i] = dy[i-1] + (dy[i-1] - 0.5) * y[i-1] + 0.5 * y[i]

    #// copy to y_sls/w_sls
    w_sls = w_sls_ref * 1. 
    y_sls = dy * y 

    return w_sls,y_sls 

@jit(nopython=True)
def compute_q_sls_model(y_sls,w_sls,om,exact=False):
    Q_ls = 1. 
    nsls = len(y_sls)
    if exact:
        for p in range(nsls):
            Q_ls += y_sls[p] * om**2 / (om**2 + w_sls[p]**2)

    # denom
    Q_demon = 0.
    for p in range(nsls):
        Q_demon += y_sls[p] * om * w_sls[p] / (om**2 + w_sls[p]**2)
    
    return Q_ls / Q_demon
@jit(nopython=True)
def get_sls_modulus_factor(freq,Q):
    om = 2 * np.pi * freq

    w_sls,y_sls = get_Q_sls_model(Q)
    s = np.sum(1j * om * y_sls / (w_sls + 1j * om))

    return s + 1.

@jit(nopython=True)
def get_sls_Q_deriv(freq,Q):
    y_sls_ref = np.array([1.93044501, 1.64217132, 1.73606189, 1.42826439, 1.66934129])
    w_sls_ref = np.array([4.71238898e-02, 6.63370885e-01, 9.42477796e+00, 1.14672436e+02,1.05597079e+03])
    dy = y_sls_ref * 0 
    y = y_sls_ref  / Q 

    # corrector
    NSLS = len(y_sls_ref)
    dy[0] = 1. + 0.5 * y[0]
    for i in range(1,NSLS):
        dy[i] = dy[i-1] + (dy[i-1] - 0.5) * y[i-1] + 0.5 * y[i]
    dd_dqi = dy * 0
    dd_dqi[0] = 0.5 * y_sls_ref[0]
    for i in range(1,NSLS):
        dd_dqi[i] = dd_dqi[i-1] + (dy[i-1] - 0.5) * y_sls_ref[i-1] + dd_dqi[i-1] * y[i-1] +  0.5 * y_sls_ref[i]
        
    dd_dqi = dd_dqi * y + dy * y_sls_ref
    om = 2 * np.pi * freq
    dsdqi = np.sum(1j * om * dd_dqi /(w_sls_ref + 1j * om))
    s = np.sum(1j * om * y * dy / (w_sls_ref + 1j * om))

    return s + 1., dsdqi

In [35]:
@jit(nopython=True)
def assemble_matrix_diag(P,Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco):
    nglob = np.max(ibool) + 1
    M = np.zeros((nglob),dtype=np.complex128)

    for ispec in range(nspec + 1):
        NGL = NGLL
        if ispec == nspec:
            wg = wgrl * 1.
            NGL = NGRL
        else:
            wg = wgll * 1.

        for i in range(NGL):
            ig = ibool[ispec*NGLL+i]
            sp = get_sls_modulus_factor(1.,Qp[ispec*NGLL+i])
            M[ig] += jaco[ispec] * wg[i] * P[ispec*NGLL+i] * sp 
    
    return M 

@jit(nopython=True)
def assemble_matrix_diag_fd(P,Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco,x,y,coef):
    dm = P * 0.0j
    for ispec in range(nspec + 1):
        NGL = NGLL
        if ispec == nspec:
            NGL = NGRL
        
        for i in range(NGL):
            id = ispec * NGLL + i
            p = P[id] * 1. 
            dx = 0.01
            P[id] = p * (1 + dx)
            M = assemble_matrix_diag(P,Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco)
            s1 = np.sum(y * M * x)

            P[id] = p * (1 - dx)
            M = assemble_matrix_diag(P,Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco)
            s2 = np.sum(y * M * x)

            dm[id] = (s1 - s2) / (p * dx * 2)

            # recover
            P[id] = p * 1. 
    
    return dm * coef


@jit(nopython=True)
def assemble_matrix_diag_deriv(Qp,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco,x,y,coef):
    dM = Qp * 0.0j

    # element
    xe = np.zeros((NGRL),dtype=np.complex128)
    ye = np.zeros((NGRL),dtype=np.complex128)

    for ispec in range(nspec + 1):
        NGL = NGLL
        if ispec == nspec:
            wg = wgrl * 1.
            NGL = NGRL
        else:
            wg = wgll * 1.

        for i in range(NGL):
            ig = ibool[ispec*NGLL+i]
            xe[i] = x[ig]
            ye[i] = y[ig]

        # compute 
        for k in range(NGL):
            sp = get_sls_modulus_factor(1.,Qp[ispec*NGLL+k])
            dM[ispec*NGLL+k] =  wg[k] * jaco[ispec] * xe[k] * ye[k] * sp * coef
    
    return dM

In [37]:
# read mesh
from scipy.io import FortranFile
fio = FortranFile("mesh.bin","r")
ibool = fio.read_ints('i4')
jaco = fio.read_reals('f8')
xN = fio.read_reals('f8')
xQN = fio.read_reals('f8')
wgll = fio.read_reals('f8')
wgrl = fio.read_reals('f8')
nspec = len(jaco) - 1
NGLL = len(wgll)
NGRL = len(wgrl)

fio.close()

np.random.seed(10)
nglob = np.max(ibool) + 1 
x = np.random.rand(nglob) + np.random.rand(nglob) * 1.0j
y = x * 1.0
coef = 1.04 + 0.04j
dM = assemble_matrix_diag_fd(xN,xQN,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco,x,y,coef)
dM1 = assemble_matrix_diag_deriv(xQN,nspec,NGLL,NGRL,ibool,wgll,wgrl,jaco,x,y,coef)

np.allclose(dM,dM1)

True

In [20]:
import numpy as np 
from scipy.linalg.lapack import dsygv
from scipy.linalg import qz,eigh

In [3]:
np.random.seed(5)
A = np.random.rand(2,2)
A = 0.5 * (A + A.T)
B = np.array([[1.,0.5],[0.5,1.2]])

In [21]:
w,Z = eigh(A,B)
AA,BB,QQ,ZZ = qz(A,B)

In [23]:
display(Z)
display(ZZ/ Z)

array([[-1.10576244,  0.2011152 ],
       [ 0.62408673,  0.81433859]])

array([[0.78757376, 2.44394418],
       [0.78757376, 1.0694194 ]])

In [33]:
A @ ZZ[:,1] - w[1] * B @ ZZ[:,1]

array([-0.15829405,  0.03909349])

In [32]:
np.linalg.norm(ZZ[:,0])

1.0

In [35]:
eigh(B)

(array([0.59009805, 1.60990195]),
 array([[-0.77334214,  0.63398891],
        [ 0.63398891,  0.77334214]]))

In [1]:
import numpy as np 

In [3]:
def lovefunc(c,r1,r2,N1,N2,L1,L2,H,om):
    gamma2 = np.sqrt((1 - r2 * c**2/N2) * N2/L2)
    Omega = om/c * H * gamma2 * (
                ((r1 * c**2 - N1) / (N2 - r2 / r1 * N1)) +
                (L2 / L1) * ((N2 - r2*c**2) / (N2 - r2/r1 * N1))
            )
    u = N1 / (c * r1) * ( c**2 * r1 / N1 + Omega) / ( 1 + Omega)
    dcc_dcL2=N1*(-1/2*H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(L2*c) + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(N2 - c**2*r2)/(L1*c*(-N1*r2/r1 + N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*((1/2)*H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(L2*c) - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(N2 - c**2*r2)/(L1*c*(-N1*r2/r1 + N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - (L2*(-1/2*N2/L2**2 + (1/2)*c**2*r2/L2**2)/(L1*np.sqrt(N2/L2 - c**2*r2/L2)) + np.sqrt(N2/L2 - c**2*r2/L2)/L1)*(N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 - H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)) + 2*c*r1/N1)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 + H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(c**2*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)))/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))
    dcc_dcL1=-H*L2*N1*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(N2 - c**2*r2)/(L1**2*c**2*r1*(-N1*r2/r1 + N2)*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + H*L2*N1*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(N2 - c**2*r2)*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(L1**2*c**2*r1*(-N1*r2/r1 + N2)*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - (-H*om*((1/2)*N1/L1**2 - 1/2*c**2*r1/L1**2)*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1)/c - ((1/2)*N1/L1**2 - 1/2*c**2*r1/L1**2)*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/np.sqrt(-N1/L1 + c**2*r1/L1) - L2*np.sqrt(N2/L2 - c**2*r2/L2)/L1**2)*(N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 - H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)) + 2*c*r1/N1)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 + H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(c**2*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)))/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))
    dcc_dcN2=N1*(H*L2*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((1/2)*(1 - c**2*r2/N2)/L2 + (1/2)*c**2*r2/(L2*N2))*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*c*(1 - c**2*r2/N2)) + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(-(-N1 + c**2*r1)/(-N1*r2/r1 + N2)**2 - L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)**2) + L2/(L1*(-N1*r2/r1 + N2)))/c)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-H*L2*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((1/2)*(1 - c**2*r2/N2)/L2 + (1/2)*c**2*r2/(L2*N2))*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*c*(1 - c**2*r2/N2)) - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(-(-N1 + c**2*r1)/(-N1*r2/r1 + N2)**2 - L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)**2) + L2/(L1*(-N1*r2/r1 + N2)))/c)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - 1/2*(N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 - H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)) + 2*c*r1/N1)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 + H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(c**2*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)))/(L1*np.sqrt(N2/L2 - c**2*r2/L2)*(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2))))
    dcc_dcN1=-H*N1*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-1/(-N1*r2/r1 + N2) + r2*(-N1 + c**2*r1)/(r1*(-N1*r2/r1 + N2)**2) + L2*r2*(N2 - c**2*r2)/(L1*r1*(-N1*r2/r1 + N2)**2))/(c**2*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(-1/(-N1*r2/r1 + N2) + r2*(-N1 + c**2*r1)/(r1*(-N1*r2/r1 + N2)**2) + L2*r2*(N2 - c**2*r2)/(L1*r1*(-N1*r2/r1 + N2)**2))/c - c**2*r1/N1**2)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) - ((1/2)*H*om*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1)/(L1*c) + (1/2)*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 - H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)) + 2*c*r1/N1)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 + H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(c**2*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)))/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2))) + (H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1))
    dcc_dr2=N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(N1*(-N1 + c**2*r1)/(r1*(-N1*r2/r1 + N2)**2) + L2*N1*(N2 - c**2*r2)/(L1*r1*(-N1*r2/r1 + N2)**2) - L2*c**2/(L1*(-N1*r2/r1 + N2)))/c - 1/2*H*c*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(N1*(-N1 + c**2*r1)/(r1*(-N1*r2/r1 + N2)**2) + L2*N1*(N2 - c**2*r2)/(L1*r1*(-N1*r2/r1 + N2)**2) - L2*c**2/(L1*(-N1*r2/r1 + N2)))/c + (1/2)*H*c*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) + (1/2)*c**2*(N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 - H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)) + 2*c*r1/N1)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 + H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(c**2*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)))/(L1*np.sqrt(N2/L2 - c**2*r2/L2)*(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2))))
    dcc_dr1=-H*N1*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-N1*r2*(-N1 + c**2*r1)/(r1**2*(-N1*r2/r1 + N2)**2) + c**2/(-N1*r2/r1 + N2) - L2*N1*r2*(N2 - c**2*r2)/(L1*r1**2*(-N1*r2/r1 + N2)**2))/(c**2*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(-N1*r2*(-N1 + c**2*r1)/(r1**2*(-N1*r2/r1 + N2)**2) + c**2/(-N1*r2/r1 + N2) - L2*N1*r2*(N2 - c**2*r2)/(L1*r1**2*(-N1*r2/r1 + N2)**2))/c + c**2/N1)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) - N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(c*r1**2*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) - (-1/2*H*c*om*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1)/L1 - 1/2*c**2*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c - H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 - H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)) + 2*c*r1/N1)/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)) + N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)*(-H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*(2*c*r1/(-N1*r2/r1 + N2) - 2*L2*c*r2/(L1*(-N1*r2/r1 + N2)))/c + H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c**2 + H*om*r2*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/(N2*(1 - c**2*r2/N2)))/(c*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)**2) - N1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + c**2*r1/N1)/(c**2*r1*(H*om*np.sqrt(N2*(1 - c**2*r2/N2)/L2)*((-N1 + c**2*r1)/(-N1*r2/r1 + N2) + L2*(N2 - c**2*r2)/(L1*(-N1*r2/r1 + N2)))/c + 1)))/(-np.sqrt(-N1/L1 + c**2*r1/L1)*(-H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c**2 + H*om*r1/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)))*(np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)**2 + 1) - c*r1*np.tan(H*om*np.sqrt(-N1/L1 + c**2*r1/L1)/c)/(L1*np.sqrt(-N1/L1 + c**2*r1/L1)) - c*r2/(L1*np.sqrt(N2/L2 - c**2*r2/L2)))

    return u,dcc_dcN1,dcc_dcN2

In [ ]:
H,r2,bh2,bv2 = 35.0, 3.2, 5.5, 5.0
r1,bh1,bv1 = 2.8,3.3,3.0
L1 = r1 * bv1**2 
L2 = r2 * bv2**2 
N1 = r1 * bh1**2 
N2 = r2 * bh2**2
om = 2 * np.pi * 1. 
c = 3.5

u,dcc_dN1,dcc_dN2 = lovefunc(c,r1,r2,N1,N2,L1,L2,H,om)

L1p = L1 * (1 + 0.01)
u1,_,_ = lovefunc(c,r1,r2,N1,N2,L1p,L2,H,om)
L1p = L1 * (1 - 0.01)
u2,_,_ = lovefunc(c,r1,r2,N1,N2,L1p,L2,H,om)

In [26]:
from scipy.io import FortranFile
import numpy as np 

In [63]:
fio = FortranFile("mat.bin","r")
S = fio.read_record('c16')
du_dx = fio.read_record('c8')
fio.close()

n = len(du_dx)
S = np.reshape(S,(n,n),order='F')

np.diag(S)

array([-20.08588409+8.52490589e-03j, -29.13581085+1.63370539e-02j,
       -16.1961422 +7.69480877e-03j, -10.44473839+5.81249502e-03j,
        -7.53049994+2.57444312e-03j,  -3.79345798+3.39113642e-04j,
        -6.34739208-1.14467144e-02j,   0.        +0.00000000e+00j,
        -7.67658234-2.88962107e-02j,  -9.04338932-4.87756655e-02j,
       -11.12021732-7.17861429e-02j, -13.51273251-9.58754867e-02j,
       -16.04442024-1.20172143e-01j, -18.65272141-1.44467264e-01j,
       -21.32124329-1.68810248e-01j, -24.05067253-1.93327487e-01j,
       -26.84672356-2.18148455e-01j, -29.7141571 -2.43369251e-01j,
       -32.65489578-2.69044578e-01j, -35.66254807-2.95146942e-01j,
       -38.71712875-3.21525753e-01j, -41.77021027-3.47784519e-01j,
       -44.72974777-3.73154700e-01j, -47.44267654-3.96349788e-01j,
       -51.4915657 -4.30832207e-01j, -49.52986526-4.14186358e-01j])

In [55]:
def solve_lower_triangular(L, b, tol=1e-12):
    """
    Solve L x = b for x where L is lower triangular.

    Parameters:
    - L: (n, n) lower triangular matrix
    - b: (n,) right-hand side vector
    - tol: small threshold to treat diagonal elements as zero

    Returns:
    - x: (n,) solution vector
    """
    n = L.shape[0]
    x = np.zeros_like(b)
    
    for i in range(n):
        if abs(L[i, i]) < tol:
            x[i] = 0. 
            continue
        
        sum_Lx = np.dot(L[i, :i], x[:i])
        x[i] = (b[i] - sum_Lx) / L[i, i]
    
    return x

In [67]:
fio = FortranFile("mat_single.bin","r")
S1 = fio.read_record('c16')
du_dx1 = fio.read_record('c8')
fio.close()

n = len(du_dx)
S1 = np.reshape(S1,(n,n),order='F')

x1 = solve_lower_triangular(S1,du_dx1,1.0e-50)

np.linalg.norm(S1 * x1 - du_dx1)


85.98343980634992

In [66]:
x = solve_lower_triangular(S,du_dx)

np.linalg.norm(S * x - du_dx)

64.92106387597505

In [54]:
S_fix = S * 1. 
sn = np.max(abs(S))
idx = abs(S) == 0.0
S_fix[idx] = 1.0e-6 * (1 + 1j) / np.sqrt(2)

spsolve_triangular(S_fix,du_dx)

/tmp/ipykernel_30251/2497934287.py:6: SparseEfficiencyWarning: CSC or CSR matrix format is required. Converting to CSC matrix.
  spsolve_triangular(S_fix,du_dx)


array([ 3.96314376e-05-1.47234725e-07j, -1.10377240e+06+1.10331359e+06j,
       -1.10377239e+06+1.10331359e+06j, -1.10377243e+06+1.10331359e+06j,
       -1.10377236e+06+1.10331359e+06j, -1.10377237e+06+1.10331359e+06j,
       -1.10377198e+06+1.10331353e+06j, -8.57758960e-01+1.59654658e-02j,
       -4.75074111e-01+1.34230553e-02j, -2.51305698e-01+1.01697871e-02j,
       -1.34951551e-01+7.87990609e-03j, -7.20346920e-02+6.49839188e-03j,
       -3.58131739e-02+5.65366983e-03j, -1.36953040e-02+5.12127416e-03j,
        4.33734920e-04+4.77615972e-03j,  9.79087248e-03+4.55558461e-03j,
        3.45213199e-03+5.02535881e-03j,  6.39876320e-03+1.76073628e-03j,
        4.44999941e-03-6.15093079e-04j,  2.30509267e-03-4.03996569e-04j,
        8.03318016e-04-3.58360149e-04j, -2.27751588e-04-3.41826081e-04j,
        4.22344040e-04-1.28318022e-04j,  1.60135194e-05+2.12654775e-06j,
        1.82489370e-04-6.33660194e-06j, -1.91661677e-14-1.91661677e-14j])

In [46]:
spsolve_triangular(S1,du_dx1)

/tmp/ipykernel_30251/695442344.py:1: SparseEfficiencyWarning: CSC or CSR matrix format is required. Converting to CSC matrix.
  spsolve_triangular(S1,du_dx1)


array([ 3.96335041e-05-1.47257136e-07j, -9.77873014e-03+3.07641762e-05j,
        8.91459143e-03-2.18715342e-05j, -2.70890560e-02+4.60923262e-05j,
        3.45962715e-02+1.01719631e-04j,  9.79626638e-02+1.20764298e-02j,
        3.42818401e-01-4.90606798e-02j, -3.89020976e+00+2.72358727e+00j,
       -6.62066952e-02+6.02275431e-01j, -5.18569685e-01+1.06000482e+00j,
       -6.47666805e-01+1.16757781e+00j, -6.48152957e-01+1.10722262e+00j,
        5.77789975e-01-9.96427803e-01j,  5.15134281e-01-8.61182410e-01j,
       -4.16872560e-01+7.55692083e-01j,  1.23578396e-01-7.28474118e-01j,
       -6.26496439e-01+8.31986111e-02j,  1.95577471e-02-5.40158670e-01j,
        2.37887880e-01-3.96079372e-01j,  2.06437372e-01-3.35534938e-01j,
       -1.42418850e-01+3.02032964e-01j, -1.23865951e-01+2.51107304e-01j,
       -1.46947143e-01+1.77018043e-01j,  8.44683221e-03-1.81975774e-01j,
       -1.67876243e-02+1.78242519e-02j, -1.15836955e-01+1.03086078e-01j])

In [ ]:
n = len(du_dx)
S1 = np.reshape(S,(n,n),order='F')
np.diag(S)

array([-2.00858707e+01+8.52490216e-03j, -2.91358166e+01+1.63370427e-02j,
       -1.61961613e+01+7.69481901e-03j, -1.04447346e+01+5.81248617e-03j,
       -7.53049898e+00+2.57449201e-03j, -3.79345512e+00+3.39355320e-04j,
       -6.34736538e+00-1.14467870e-02j,  1.78813934e-07+5.07570803e-08j,
       -7.67656565e+00-2.88962163e-02j, -9.04317284e+00-4.87744994e-02j,
       -1.11201191e+01-7.17855170e-02j, -1.35129290e+01-9.58763510e-02j,
       -1.60446320e+01-1.20173492e-01j, -1.86530647e+01-1.44466728e-01j,
       -2.13210812e+01-1.68810919e-01j, -2.40515289e+01-1.93336338e-01j,
       -2.68457127e+01-2.18142897e-01j, -2.97160492e+01-2.43393958e-01j,
       -3.26556511e+01-2.69056320e-01j, -3.56639061e+01-2.95155764e-01j,
       -3.87195892e+01-3.21539998e-01j, -4.17674751e+01-3.47769499e-01j,
       -4.47245750e+01-3.73107374e-01j, -4.74581184e+01-3.96485329e-01j,
       -5.14868240e+01-4.30799901e-01j, -4.95219803e+01-4.14125562e-01j])